In [1]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime


In [14]:
startdate = datetime.datetime(2022,3,28)
enddate = datetime.datetime(2022,4,1)
st = pdr.get_data_yahoo('MMM',start=startdate,end=enddate)
st

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-28,150.130005,147.190002,149.929993,149.470001,2679900,149.470001
2022-03-29,152.919998,150.039993,151.000000,152.080002,2693200,152.080002
2022-03-30,153.259995,150.770004,151.330002,151.429993,2395100,151.429993
2022-03-31,151.250000,148.800003,150.449997,148.880005,3193800,148.880005
2022-04-01,149.860001,146.850006,149.630005,149.690002,2196400,149.690002


In [26]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'aR8jQP7cfr623h9apTvM711pm'
consumer_secret = 'SLKF8TOUL8PtapZ5amz4Y9ZBRE99ocRxwvEnzIjfEFri2TbhTI'
access_key= '1511751059332493317-eFiU4zDgdWqZPDC7EWZrQVaCLJ5ZX8'
access_secret = 'U3yAxDcXSFVHqsbJW1adpZiElFoIBosY7FW1UJpAOULqM'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

query = "$"+"MMM" + " -filter:retweets -filter:replies"
tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', since = startdate, until = enddate, tweet_mode='extended').items(5)
print(tweets)
for tweet in tweets:
    status=tweet._json
    created_at = status['created_at']
    tweet_id = status['id_str']
    tweet_text = status['full_text']
    tweet_text = re.sub(r'\W', ' ',tweet_text)
    print(created_at)

Unexpected parameter: since


Thu Mar 31 22:51:05 +0000 2022
Thu Mar 31 22:30:02 +0000 2022
Thu Mar 31 20:28:37 +0000 2022
Thu Mar 31 17:33:06 +0000 2022
Tue Mar 29 23:03:05 +0000 2022


In [22]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
tickers1 = tickers[:99]
tickers2 = tickers[99:199]
tickers3 = tickers[199:299]
tickers4 = tickers[299:399]
tickers5 = tickers[399:499]
tickers6 = tickers[499:]
print(tickers2)

['CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FAST', 'FRT', 'FDX', 'FITB', 'FRC']


In [4]:
# Get stock prices and returns
startdate = datetime.datetime(2022,3,21)
enddate = datetime.datetime(2022,3,25)

file = open("stocks.csv",'w',newline='')
print(len(tickers))
count = 0
for ticker in tickers:
    count +=1
    print(count)
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass



505
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [ ]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'ELRou1VKpS5p7EBEGamXqqGRT'
consumer_secret = 'QbsToHJOyA9ZsDrFCCw1aErF9LubwqlfLpLcJviruhKepQwX3D'
access_key= '3138259520-6jOClvoHXzF5DCf2XaylTxhrn1Rb7X50ykQyU3C'
access_secret = 'r9bokM5UvRbS8QnsYdQksSi2tO8gRj7T4sfq2K41jzmQM'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets4.csv","w",newline='')
alltweets = csv.writer(file)
count = 0
for ticker in tickers5:
    count += 1
    print(count)
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', until = enddate, tweet_mode='extended').items(10)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass




1
Realty Income Corporation current quarter revenue growth YoY of 63 00  ranked 742 out of 5722 companies in our database      https   t co e5GlDDjC8K  O
2
   REG price target update    Target Lowered by Deutsche Bank Aktiengesellschaft From  92 00 to  83 00  Day quote change   58 76   1 45   Target upside  41 25   Published  March 31  2022 https   t co RJsd063rBB
3
 REGN   SEC Form 4  Brown Michael S exercised 1 930 shares at a strike of  374 12 and sold  1 349 070 worth of shares  1 930 units at  699 00  as part of a pre agreed trading plan  https   t co D2EosknQkz  45 seconds delayed 
New Large Cap 52  Week Highs for Mar 29  2022    UNP  276 82  AZN  66 60  ANTM  492 22  CB  218 99  DUK  110 99  CP  83 26  SO  72 07  REGN  699 00  AON  329 46  D  84 90  Research on https   t co jdIQT7KKID
4


In [39]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)

sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)

for row in alltweets:
    blob = TextBlob(row[3])
    if blob.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive"])
    elif blob.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative"])     
    elif blob.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral"])    
        
file.close()
sentiment_file.close()

Error: line contains NUL